Make injections according to the `IASPrior` in batches of $\mathcal{M}$, restricted to $\langle h \mid h \rangle > 60$
<div class="alert alert alert-block alert-warning">
    Eventually a lot of files will be saved inside a directory <tt>data/</tt>. If you would rather have them somewhere else (e.g. in a different partition) you can make a symlink:

    ln -s <directory_where_to_save_injections> data
</div>

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import sys
sys.path.append('../..')

import pandas as pd

from cogwheel.validation import config
from cogwheel.validation import generate_injections

from cogwheel import gw_plotting

In [ ]:
overwrite = False  # Change this if you want

for inj_dir, mchirp_range in zip(config.INJECTION_SET_DIRS, config.MCHIRP_RANGES):
    if not overwrite and inj_dir.exists():
        print('Skipping existing', inj_dir.as_posix())
        continue

    print(f'Generating injections with mchirp in {mchirp_range}...')
    injections_above_threshold \
        = generate_injections.generate_injections_above_threshold(mchirp_range)

    inj_dir.mkdir(exist_ok=overwrite)
    injections_above_threshold.to_feather(inj_dir/config.INJECTIONS_FILENAME)

In [ ]:
for inj_dir, mchirp_range in zip(config.INJECTION_SET_DIRS, config.MCHIRP_RANGES):
    injections_above_threshold = pd.read_feather(inj_dir/config.INJECTIONS_FILENAME)
    
    # Plot ⟨ℎ∣ℎ⟩ distribution:
    generate_injections.test_h_h_distribution(injections_above_threshold)

    # Corner plot:
    params = [
        'm1',
        'm2',
        's1z',
        's2z',
        's1x_n',
        's1y_n',
        's2x_n',
        's2y_n',
        'iota',
        'ra',
        'dec',
        'psi',
        'phi_ref',
        'd_luminosity']

    gw_plotting.CornerPlot(injections_above_threshold[params]).plot(
        title=rf'$\mathcal{{M}} \in {mchirp_range}\,\rm{{M}}_\odot$')